<a href="https://colab.research.google.com/github/Rogerio-mack/VLF/blob/main/VLF_create_json_from_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This script

Para todos os compartilhamentos do Google Drive do usuário `client_id | client_secret` gera um `json_VLF.json` do tipo:

```
dict_files[collection][dir_year][dir_type][dir_station][dir_date][fname] = {'id': row['id'],'mimeType': row['mimeType'],'path': row['path']}
```

Para todos os arquivos VLF no formato awesome. Um exemplo de saída: 

```
VLF:
  '2007':
    narrowband:
      FE:
        '2007_07_10':
          FE070710000500NAA_006A.mat:
            id: 1FuieXUCB0x2s-Wvej3di1eFkcy_KP1_E
            mimeType: application/octet-stream
            path: Not Available
          FE070710000500NAA_006B.mat:
            id: 1yGs4BVV6RD1SihCNccobqRw-YPhWZlkq
            mimeType: application/octet-stream
            path: Not Available
    broadband:
      SP:
        ...

  '2011':
    broadband:
      SP:
        '2011_11_26':
          SP111126000000_001.mat:
            id: 1ItILI1UTvjVgzcbRSqjCx4D-A5aBjNCT
            mimeType: application/octet-stream
            path: Not Available
          ...
          SP111126001500_001.mat:
            id: 1iunzZuR6XYN0Kb5LOPWA3MfGC7dvtf54
            ...
      TA:
        '2011_01_02':
          TA110102032412_000.mat:
            id: 1-TOGJArb62-wqAqC6CW3cjt5N5AdsBrI
            mimeType: application/octet-stream
            path: Not Available
      ...
    'end_point_type': GoogleDrive  
```

# Install

In [ ]:
!pip install names

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Imports

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import shutil
from google.colab import drive

import yaml
import json
import datetime 
import time
import random
import names

import shutil
import time
from tqdm.auto import tqdm

# Google Drive Set Up



## Google Drive API imports

In [ ]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload


## Google Drive SCOPES and client secret

In [ ]:
import getpass

client_id = getpass.getpass('Enter your client_id:')
client_secret = getpass.getpass('Enter your client_secret:')

Enter your client_id:··········
Enter your client_secret:··········


In [ ]:
SCOPES = ["https://www.googleapis.com/auth/drive"]

client_secret = {"installed":{"client_id":client_id,
                              "project_id":"projectvlf","auth_uri":"https://accounts.google.com/o/oauth2/auth",
                              "token_uri":"https://oauth2.googleapis.com/token",
                              "auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs",
                              "client_secret":client_secret,
                              "redirect_uris":["http://localhost"]}}

with open("client_secret.json", "w") as outfile:
    json.dump(client_secret, outfile)

# `get_Google_credentials(client_secret_file="client_secret.json", SCOPES=SCOPES)`

And create drive instance.

In [ ]:
def get_Google_credentials(client_secret_file="client_secret.json", SCOPES=SCOPES):

  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.

  if os.path.exists("token.json"):
      creds = Credentials.from_authorized_user_file("token.json", SCOPES)

  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
      if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
      else:
          flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
          creds = flow.run_console()  # run_local_server(port=0)
      
      # Save the credentials for the next run
      with open("token.json", "w") as token:
          token.write(creds.to_json())

  return creds

creds = get_Google_credentials()
service = build("drive", "v3", credentials=creds)

# `create_Google_df_files(service)`

In [ ]:
def create_Google_df_files(service):

  page_size = 500
  query = "name contains '.mat' and mimeType = 'application/octet-stream'"
  fields = "nextPageToken, files(id, name, mimeType)"

  data = service.files().list(pageSize=page_size, 
                              q = query,
                              fields=fields, 
                              includeItemsFromAllDrives=False, 
                              supportsAllDrives=True).execute()

  df = pd.DataFrame()
  df = pd.concat([df,pd.DataFrame(data['files'])])
  df = df.reset_index(drop=True)
  
  print(len(df), ' files processed')

  while 'nextPageToken' in data.keys():

    data = service.files().list(pageToken=data['nextPageToken'], pageSize=500, 
                                q = query,
                                fields=fields, 
                                includeItemsFromAllDrives=False, 
                                supportsAllDrives=True).execute()

    df = pd.concat([df,pd.DataFrame(data['files'])])
    df = df.reset_index(drop=True)

    print(len(df), ' files processed')

  df.rename(columns={'name':'fname'},inplace=True)
  df = df[['fname', 'id', 'mimeType']]

  return df

df_files = create_Google_df_files(service)
display(df_files)

500  files processed
1000  files processed
1500  files processed
2000  files processed
2500  files processed
3000  files processed
3500  files processed
4000  files processed
4500  files processed
5000  files processed
5500  files processed
6000  files processed
6500  files processed
7000  files processed
7500  files processed
8000  files processed
8500  files processed
9000  files processed
9500  files processed
10000  files processed
10500  files processed
11000  files processed
11500  files processed
12000  files processed
12500  files processed
13000  files processed
13500  files processed
14000  files processed
14500  files processed
15000  files processed
15500  files processed
16000  files processed
16500  files processed
17000  files processed
17500  files processed
18000  files processed
18500  files processed
19000  files processed
19500  files processed
20000  files processed
20500  files processed
21000  files processed
21500  files processed
22000  files processed
22500  f

,fname,id,mimeType
0,JU210806000000_001.mat,1CVIQl8by0ktHVN7sKjifD3EGpKo8OZrq,application/octet-stream
1,TA210908000000_000.mat,16dE9mL4_JSN0-2PAwnP9xmlExHDtSWKE,application/octet-stream
2,JU200806000000_000.mat,12ScpslapIOhtKXHeZybNnxlKOKHQLDHy,application/octet-stream
3,JU220806000000_001.mat,1CGHJEPailQwb6Pow-RhchNcYUS8Lxjhb,application/octet-stream
4,JU210806000000_000.mat,1COp282mprC9kCXY-XGXc9WbofdQ3A7ZV,application/octet-stream
...,...,...,...
24519,SP111126000500_001.mat,1g43X5PpkWZ6pHhhPjVfKuEl2rf-3CHkS,application/octet-stream
24520,SP111126000500_000.mat,1gbf4C9tIG30bd5IGCLSCieyGKi-M89l5,application/octet-stream
24521,TA110102032500_000.mat,1v4jtQYADfNiD5oD3SkfImtLoH5EwHt13,application/octet-stream
24522,TA110102032412_001.mat,1i5BN5kc3FM1XzOovgEMnG6YwGC3haxdq,application/octet-stream


# `get_Google_path(fileId)`

In [ ]:
def get_Google_path(fileId):

  tree = []   
  file = service.files().get(fileId=fileId, fields='id, name, parents').execute()
  parent = file.get('parents')
  if parent:
      while True:
          folder = service.files().get(
              fileId=parent[0], fields='id, name, parents').execute()
          parent = folder.get('parents')
          if parent is None:
              break
          # tree.append({'id': parent[0], 'name': folder.get('name')})
          tree.append('/' + folder.get('name'))

  tree.reverse()
  path = ''
  for d in tree:
    path = path + d

  return path

paths = []  
for _, row in df_files.head().iterrows():            # We really don't need this, so just get a few...  
  paths.append( get_Google_path(row['id']) )

paths

['/2021/broadband/Juneau/2021_08_06',
 '/CRAAM_VLF/2021/broadband/Taylor/2021_09_08',
 '/CRAAM_VLF/2020/broadband/Juneau/2020_08_06',
 '/2022/broadband/Juneau/2022_08_06',
 '/2021/broadband/Juneau/2021_08_06']

In [ ]:
df_files['path'] = 'Not Available'
display(df_files)

,fname,id,mimeType,path
0,JU210806000000_001.mat,1CVIQl8by0ktHVN7sKjifD3EGpKo8OZrq,application/octet-stream,Not Available
1,TA210908000000_000.mat,16dE9mL4_JSN0-2PAwnP9xmlExHDtSWKE,application/octet-stream,Not Available
2,JU200806000000_000.mat,12ScpslapIOhtKXHeZybNnxlKOKHQLDHy,application/octet-stream,Not Available
3,JU220806000000_001.mat,1CGHJEPailQwb6Pow-RhchNcYUS8Lxjhb,application/octet-stream,Not Available
4,JU210806000000_000.mat,1COp282mprC9kCXY-XGXc9WbofdQ3A7ZV,application/octet-stream,Not Available
...,...,...,...,...
24519,SP111126000500_001.mat,1g43X5PpkWZ6pHhhPjVfKuEl2rf-3CHkS,application/octet-stream,Not Available
24520,SP111126000500_000.mat,1gbf4C9tIG30bd5IGCLSCieyGKi-M89l5,application/octet-stream,Not Available
24521,TA110102032500_000.mat,1v4jtQYADfNiD5oD3SkfImtLoH5EwHt13,application/octet-stream,Not Available
24522,TA110102032412_001.mat,1i5BN5kc3FM1XzOovgEMnG6YwGC3haxdq,application/octet-stream,Not Available


# Generate `name_conventions.json` and `name_conventions` `dict`

In [ ]:
# Narrowband Files
#-------------------------------------------------------------------------------
# Filename: XXYYMMDDHHMMSSZZZ_ACCT.mat

# XX – Station ID
# YY – Year
# MM — Month
# DD — Day
# HH — Hour
# MM — Minute
# SS — Second
# ZZZ — Transmitter Callsign
# A — Usually not used
# CC — 00 for N/S channel, 01 for E/W channel
# T — Type of data
# A is low resolution (1 Hz sampling rate) amplitude
# B is low resolution (1 Hz sampling rate) phase
# C is high resolution (50 Hz sampling rate) amplitude
# D is high resolution (50 Hz sampling rate) phase
# F is high resolution (50 Hz sampling rate) effective group delay

# XXYYMMDDHHMMSSZZZ_ACCT.mat
# 01234567890123456789012345

# Braoadband Files
#-------------------------------------------------------------------------------
# Filename: XXYYMMDDHHMMSS_ACC.mat

# XX – Station ID
# YY – Year
# MM — Month
# DD — Day
# HH — Hour
# MM — Minute
# SS — Second
# A — Sampling rate. 0 for 100 kHz sampled data (VLF), 1 for 1 MHz sampled data (LF), 2 for 25 kHz sampled data (Siple station experiment).
# CC — 00 for N/S channel, 01 for E/W channel

# XXYYMMDDHHMMSS_ACC.mat
# 0123456789012345678901 

name_conventions = {
    
'narrowband' : {    
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  'Transmitter':      [14,16],
  '_':                [17,17],
  'Usually not used': [18,18],
  'CC':               [19,20],
  # CC — 00 for N/S channel, 01 for E/W channel
  'Type_ABCDF':       [21,21],
  # A is low resolution (1 Hz sampling rate) amplitude
  # B is low resolution (1 Hz sampling rate) phase
  # C is high resolution (50 Hz sampling rate) amplitude
  # D is high resolution (50 Hz sampling rate) phase
  # F is high resolution (50 Hz sampling rate) effective group delay
  '.mat':             [22,25]
  },

'broadband' : { 
  'Station_ID':       [0,1],
  'Year':             [2,3],
  'Month':            [4,5],
  'Day':              [6,7],
  'Hour':             [8,9],
  'Minute':           [10,11],
  'Second':           [12,13],
  '_':                [14,14],
  'A':                [15,15],
  # A — Sampling rate. 0 for 100 kHz sampled data (VLF), 1 for 1 MHz sampled data (LF), 2 for 25 kHz sampled data (Siple station experiment). 
  'CC':               [16,17],
  # CC — 00 for N/S channel, 01 for E/W channel
  '.mat':             [18,21]
  }  

}

f = open("name_conventions.json", "w")
f.write(json.dumps(name_conventions))
f.close()

with open("name_conventions.json") as json_file:
  name_conventions = json.load(json_file)

print(name_conventions)
print(name_conventions['narrowband'])

{'narrowband': {'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], 'Transmitter': [14, 16], '_': [17, 17], 'Usually not used': [18, 18], 'CC': [19, 20], 'Type_ABCDF': [21, 21], '.mat': [22, 25]}, 'broadband': {'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], '_': [14, 14], 'A': [15, 15], 'CC': [16, 17], '.mat': [18, 21]}}
{'Station_ID': [0, 1], 'Year': [2, 3], 'Month': [4, 5], 'Day': [6, 7], 'Hour': [8, 9], 'Minute': [10, 11], 'Second': [12, 13], 'Transmitter': [14, 16], '_': [17, 17], 'Usually not used': [18, 18], 'CC': [19, 20], 'Type_ABCDF': [21, 21], '.mat': [22, 25]}


# `get_from_name(fname, field, type_band='narrowband',...)`

In [ ]:
def get_from_name(fname, field, type_band='narrowband', name_conventions=name_conventions):
  
  return fname[ int(name_conventions[type_band][field][0]) : int(name_conventions[type_band][field][1]) + 1 ]

with open("name_conventions.json") as json_file:
  name_conventions = json.load(json_file)
      
fname = 'AL110505000000NAA_101A.mat'
print(get_from_name(fname,'Station_ID'))

for key in name_conventions['narrowband'].keys():
  print(key,' : ', get_from_name(fname,key))

AL
Station_ID  :  AL
Year  :  11
Month  :  05
Day  :  05
Hour  :  00
Minute  :  00
Second  :  00
Transmitter  :  NAA
_  :  _
Usually not used  :  1
CC  :  01
Type_ABCDF  :  A
.mat  :  .mat


# `what_type(path, fname)`

In [ ]:
import datetime 

def what_type(path, fname):
# narrowband 
# XXYYMMDDHHMMSSZZZ_ACCT.mat
# 01234567890123456789012345

# broadband
# XXYYMMDDHHMMSS_ACC.mat
# 0123456789012345678901 

  if fname.split('.')[-1] != 'mat': return None
  if fname.count('_') != 1: return None
  if fname.count('.') != 1: return None

  try:
    datetime.datetime(int('20' + get_from_name(fname,'Year')),
                      int(get_from_name(fname,'Month')),
                      int(get_from_name(fname,'Day')),
                      int(get_from_name(fname,'Hour')),
                      int(get_from_name(fname,'Minute')),
                      int(get_from_name(fname,'Second')))
  except:
    return None

  for type_band in ['narrowband','broadband']:
    if (fname.find('_') ==  name_conventions[type_band]['_'][0]) and (fname.find('.mat') ==  name_conventions[type_band]['.mat'][0]):
      return type_band  

  return None

print('ok')
print( what_type(None,'MH210113000000DHO_101A.mat') )
print( what_type(None,'CH200721053000_001.mat') )
print( what_type(None,'CH000721053000_001.mat') )
print()

print('nok')
print( what_type(None,'CH0a0721053000_001.mat') )
print( what_type(None,'CH000721053060_001.mat') )
print( what_type(None,'CH222021053000_001.mat') )
print( what_type(None,'MH210160000000DHO_101A.mat') )
print( what_type(None,'MH210113000060DHO_101A.mat') )
print( what_type(None,'MH210140000000DHO_101A.mat') )
print( what_type(None,'teste1.mat') )
print( what_type(None,'teste2_asd.mat') )
print( what_type(None,'teste2_asd.csv') )

ok
narrowband
broadband
broadband

nok
None
None
None
None
None
None
None
None
None


## Set type narrowband, broadband or None
Exclude None types

In [ ]:
df_files['type'] = df_files.fname.apply(lambda x: what_type(None,x))

print( len(df_files[   df_files['type'].isnull() ]), ' None files excluded.' )
df_files = df_files[ - df_files['type'].isnull() ]
df_files = df_files.reset_index(drop=True)
df_files.head()

print( len(df_files), ' Valid files.' )

13  None files excluded.
24511  Valid files.


# `create_keys(df_files, collection='VLF', dict_stations=None)`

In [ ]:
def create_keys(df_files, collection='VLF', end_point_type = 'GoogleDrive', dict_stations=None):

  dict_files = {}

  if collection     not in dict_files.keys():
    dict_files[collection] = {}

  if end_point_type not in dict_files[collection].keys():
    dict_files[collection]['end_point_type'] = end_point_type

  pbar = tqdm(desc='Files processing', total=len(df_files))

  for ix, row in df_files.iterrows():
    pbar.update(1)

    fname = row['fname']

    dir_type = what_type(None,fname)

    if dir_type not in ['narrowband','broadband']:
      # sys.exit('There is an inconsistent type_band in df_files. ' + str(fname) + ' is type ' + str(dir_type))
      print('There is an inconsistent type_band in df_files. ' + str(fname) + ' is type ' + str(dir_type))
      continue

    dir_year =  '20' + get_from_name(fname,'Year')

    dir_date =  '20' + get_from_name(fname,'Year') + '_' + \
                get_from_name(fname,'Month') + '_' + \
                get_from_name(fname,'Day')

    dir_station = get_from_name(fname,'Station_ID')

    if dict_stations is not None:
      if dir_station in dict_stations.keys():
        dir_station = dict_stations[dir_station]

    if dir_year       not in dict_files[collection].keys():
      dict_files[collection][dir_year] = {}

    if dir_type       not in dict_files[collection][dir_year].keys():
      dict_files[collection][dir_year][dir_type] = {}

    if dir_station    not in dict_files[collection][dir_year][dir_type].keys():
      dict_files[collection][dir_year][dir_type][dir_station] = {}

    if dir_date       not in dict_files[collection][dir_year][dir_type][dir_station].keys():
      dict_files[collection][dir_year][dir_type][dir_station][dir_date] = {}

    dict_files[collection][dir_year][dir_type][dir_station][dir_date][fname] = {'id': row['id'],  
                                                                                'mimeType': row['mimeType'], 
                                                                                'path': row['path']  }
  
  return dict_files

dict_files = create_keys(df_files, collection='VLF', dict_stations=None)                     


Files processing:   0%|          | 0/24511 [00:00<?, ?it/s]

In [ ]:
print( list(dict_files) )
print( list(dict_files['VLF']) )
print( dict_files['VLF']['end_point_type'] )
print( list(dict_files['VLF']['2011']) )
print( list(dict_files['VLF']['2011']['narrowband']) )
print( list(dict_files['VLF']['2011']['narrowband']['PA']) )
print( list(dict_files['VLF']['2011']['narrowband']['PA']['2011_05_26']) )
print(yaml.dump(dict_files['VLF']['2011']['narrowband']['PA']['2011_05_26']) )

['VLF']
['end_point_type', '2021', '2020', '2022', '2011', '2007', '2015', '2014']
GoogleDrive
['broadband', 'narrowband']
['PA', 'SP']
['2011_05_26', '2011_01_05']
['PA110526000000GBZ_100A.mat', 'PA110526000000DHO_101D.mat', 'PA110526000000DHO_100D.mat', 'PA110526000000DHO_101C.mat', 'PA110526000000DHO_100C.mat', 'PA110526000000DHO_100F.mat', 'PA110526000000DHO_101F.mat', 'PA110526000000DHO_100A.mat', 'PA110526000000DHO_100B.mat', 'PA110526000000DHO_101A.mat', 'PA110526000000DHO_101B.mat']
PA110526000000DHO_100A.mat:
  id: 136xbdjPigBuLuADzie0YXdzY6aGgp_xL
  mimeType: application/octet-stream
  path: Not Available
PA110526000000DHO_100B.mat:
  id: 1GLsVWwNms5WLbdMWm673TLtMfkBrNa4p
  mimeType: application/octet-stream
  path: Not Available
PA110526000000DHO_100C.mat:
  id: 1fIk64CxOOx2DnUANFUwHJmNUw63JS2Rk
  mimeType: application/octet-stream
  path: Not Available
PA110526000000DHO_100D.mat:
  id: 10BcaGxopn9kYdEYkvSCJzbSsJCspPu5D
  mimeType: application/octet-stream
  path: Not Availa

In [ ]:
json_dump = json.dumps(dict_files)

with open("json_VLF.json", "w") as outfile:
  outfile.write(json_dump)
outfile.close()

f = open("json_VLF.json", "r") 
print(f.read(1000))

{"VLF": {"end_point_type": "GoogleDrive", "2021": {"broadband": {"JU": {"2021_08_06": {"JU210806000000_001.mat": {"id": "14vXwZIVgb53qitlJLOtVa3S09EaqVB97", "mimeType": "application/octet-stream", "path": "Not Available"}, "JU210806000000_000.mat": {"id": "14iaDbzVxmu0-3wV6tKfvVGC4nIexBC1H", "mimeType": "application/octet-stream", "path": "Not Available"}}, "2021_08_05": {"JU210805214530_000.mat": {"id": "14oBYXIIHvD1DtsV0iHH7-CbdNy0s2uIf", "mimeType": "application/octet-stream", "path": "Not Available"}, "JU210805214530_001.mat": {"id": "1CKBszFBPXBZGCNDd87KfNzJRaXDTb6LS", "mimeType": "application/octet-stream", "path": "Not Available"}}}, "TA": {"2021_09_08": {"TA210908000000_000.mat": {"id": "1DKjZCf8PKG8xSpxq1scbm6lm1V30v8vt", "mimeType": "application/octet-stream", "path": "Not Available"}, "TA210908000000_001.mat": {"id": "1D-XAMOLLnrOTNFgkExNbD7mvoeb0pot9", "mimeType": "application/octet-stream", "path": "Not Available"}}, "2021_06_03": {"TA210603000000_000.mat": {"id": "1DfSY0p

In [ ]:
'''
Use print(f.read(1000)) instead:

!cat json_VLF.json

Because return a error:

The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

'''

'\nUse print(f.read(1000)) instead:\n\n!cat json_VLF.json\n\nBecause return a error:\n\nThe notebook server will temporarily stop sending output\nto the client in order to avoid crashing it.\nTo change this limit, set the config variable\n`--NotebookApp.iopub_data_rate_limit`.\n\nCurrent values:\nNotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)\nNotebookApp.rate_limit_window=3.0 (secs)\n\n'

In [ ]:
yaml_dump = yaml.dump(dict_files, default_flow_style=False) 

In [ ]:
yaml_dump = yaml.dump(dict_files, default_flow_style=False) 
print(yaml_dump[0:5000]) # too much to print

VLF:
  '2007':
    narrowband:
      FE:
        '2007_07_10':
          FE070710000500NAA_006A.mat:
            id: 1FuieXUCB0x2s-Wvej3di1eFkcy_KP1_E
            mimeType: application/octet-stream
            path: Not Available
          FE070710000500NAA_006B.mat:
            id: 1yGs4BVV6RD1SihCNccobqRw-YPhWZlkq
            mimeType: application/octet-stream
            path: Not Available
  '2011':
    broadband:
      SP:
        '2011_11_26':
          SP111126000000_001.mat:
            id: 1ItILI1UTvjVgzcbRSqjCx4D-A5aBjNCT
            mimeType: application/octet-stream
            path: Not Available
          SP111126000500_000.mat:
            id: 1gbf4C9tIG30bd5IGCLSCieyGKi-M89l5
            mimeType: application/octet-stream
            path: Not Available
          SP111126000500_001.mat:
            id: 1g43X5PpkWZ6pHhhPjVfKuEl2rf-3CHkS
            mimeType: application/octet-stream
            path: Not Available
          SP111126001000_000.mat:
            id: 1XigXUb

# Download json

In [ ]:
from google.colab import files

files.download('json_VLF.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>